<a href="https://colab.research.google.com/github/edproudlove/congenial-spork/blob/main/Genetic_algos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
